## SUV (Standard Uptake Value) Calculations

In [ ]:
# extract half life and start time from header
RefDs = dicom.read_file(lstFilesDCM[0], force=True)

radSeq = RefDs.RadiopharmaceuticalInformationSequence
halfLife = float(radSeq[0]['RadionuclideHalfLife'].value) #seconds
startScan = radSeq[0]['RadiopharmaceuticalStartDateTime'].value
startScan = startScan[:-5]
startScan = datetime.strptime(startScan, '%Y%m%d%H%M%S')

print(startScan)

In [ ]:
# receive input (dose 1, time 1, dose 2, time 2, weight)
print('Enter syringe 1 dose amount (MBq): ')
dose1 = float(input())

print ('Enter syringe 1 start time (YYYY-MM-DD hh:mm): ')
start1 = input()
start1 = datetime.strptime(start1, '%Y-%m-%d %H:%M')

print('Enter syringe 2 dose amount (MBq): ')
dose2 = float(input())

print ('Enter syringe 2 start time (YYYY-MM-DD hh:mm): ')
start2 = input()
start2 = datetime.strptime(start2, '%Y-%m-%d %H:%M')

print ('Enter body weight (g): ')
weight = float(input())

In [ ]:
# calculate decay correction
# all time in minutes, doses in MBq

halfLifeMin = halfLife / 60 #halfLife seconds to minutes
timeDiff1o = start1 - startScan
timeDiff1 = np.abs(int(timeDiff1o.total_seconds() / 60)) #take negative or absolute?
timeDiff1n = int(timeDiff1o.total_seconds() / 60)
timeDiff2 = start2 - startScan
timeDiff2 = int(timeDiff2.total_seconds() / 60)

decayConst = np.log(2)/halfLifeMin

D1 = dose1 * np.exp((decayConst * -1) * (timeDiff1n))
D2 = dose2 * np.exp(decayConst * (timeDiff2))

decayCorr = D1 - D2

ArrayMBq = og_arr / 1000 #ArrayDicom is in kBq/cc. change to MBq/cc

In [ ]:
# calculate SUV
 
ArraySUV = ArrayMBq / (decayCorr / weight)
#print(decayCorr/weight)

In [ ]:
f,axarr = plt.subplots(1,2)
a = axarr[0].imshow(ArraySUV[:,:,70,20], vmin=0,vmax=1.6)
b = axarr[1].imshow(ans_suv_arr[:,:,70,20],vmin=0,vmax=1.6)
plt.colorbar(a,cax=f.add_axes([0.92,0.2,0.02,0.6]))
plt.title('Calculated SUV Slice vs. Original SUV Slice', horizontalalignment='right')

In [ ]:
suv_wb_nii = nb.Nifti1Image(ArraySUV, og_image.affine)
print(suv_wb_nii.shape) # test
suv_wb_nii_first = image.index_img(suv_wb_nii, 20) # visualize one instance
print(suv_wb_nii_first.shape) 

In [ ]:
# Saver

print('Save? (Y/N): ')
inp = input().capitalize()
if inp =='Y':
  suv_name = sample_og_name.replace('PET', 'PET_SUV')
  nb.save(suv_wb_nii, rslt_pth + suv_name)
  print(suv_name + 'saved in ' + rslt_pth)

In [ ]:
# visualize

plotting.plot_stat_map(suv_br_nii_first, bg_img=False, colorbar=True, cmap='Spectral', black_bg=True)

# PET Scan Averaging

In [ ]:
# set Answer PATHs

sample_og_name = '20201111_WB_No1_FDG_PET.nii'
sample_og = ans_pth + sample_og_name
ans_suv = ans_pth + '20201111_WB_No1_FDG_PET_SUV.nii'
ans_avg = ans_pth + '20201111_WB_No1_FDG_PET_avg40-90min.nii'

In [ ]:
# READ .NII FILES AND DICOM HEADERS
og_image = nb.load(sample_og) #original nii file
og_arr = np.array(og_image.dataobj)

ans_suv_im = nb.load(ans_suv) #suv nii file
ans_suv_arr = np.array(ans_suv_im.dataobj)

ans_avg_im = nb.load(ans_avg) #40-90min avg nii file
ans_avg_im_squeezed = nb.squeeze_image(ans_avg_im) # truncate curtailing 1
ans_avg_arr = np.array(ans_avg_im_squeezed.dataobj)

print('og_arr:', og_arr.shape)
print('ans_suv_arr:', ans_suv_arr.shape)
print('ans_avg_arr:', ans_avg_arr.shape)

In [ ]:
# put all .dcm files in Path into lstFilesDCM

PathDicom = pet_pth
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
      #if "._" in filename.lower():   # remove ghost files if existant
        #os.remove(PathDicom + )
      
      if "._" not in filename.lower():  # check whether the file's DICOM. In our case, none DICOM files start in '._'   
        lstFilesDCM.append(os.path.join(dirName,filename))

In [ ]:
# test
len(lstFilesDCM)

In [ ]:
# Get ref file
RefDs = dicom.read_file(lstFilesDCM[0], force=True)

# Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM))

# Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
z = np.arange(0.0, (RefDs.NumberOfSlices+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])
t = RefDs.NumberOfTimeSlices

In [ ]:
# test
len(RefDs)

In [ ]:
# time(ms) array
time2dur = {}
imgPosZ = np.zeros(len(lstFilesDCM))
refTime = np.zeros(len(lstFilesDCM))
slopes = np.zeros(len(lstFilesDCM))

# initialize pixel arrays
RawDicom = np.zeros((len(x)-1, len(y)-1, RefDs.NumberOfSlices,t))
RSDicom = np.zeros((len(x)-1, len(y)-1, RefDs.NumberOfSlices,t))

# iterate over dicom files and collect needed header & pixel data
for i in range(len(lstFilesDCM)):
  currRef = dicom.read_file(lstFilesDCM[i], force=True)
  
  # Store z position, reference time, frame duration
  imgPosZ[i] = currRef.ImagePositionPatient[2]
  refTime[i] = currRef.FrameReferenceTime
  time2dur[currRef.FrameReferenceTime] = currRef.ActualFrameDuration
  slopes[i] = currRef.RescaleSlope

zVals = np.unique(imgPosZ) #These are all of the z axis positions. There are 163
tVals = np.unique(refTime) #There are all of the time frames. There are 41


In [ ]:
# test: check outputs

print('zVals:', zVals.shape)
print('tVals:', tVals.shape)

In [ ]:
RawDicom = np.zeros((len(x)-1, len(y)-1, RefDs.NumberOfSlices,t))
RSDicom = np.zeros((len(x)-1, len(y)-1, RefDs.NumberOfSlices,t))

# iterate again for pixel arrays
for i in range(len(lstFilesDCM)):
  currRef = dicom.read_file(lstFilesDCM[i], force=True)
  
  #find z position and reference time
  currZ = int(np.where(zVals==currRef.ImagePositionPatient[2])[0])
  currT = int(np.where(tVals == currRef.FrameReferenceTime)[0][0])

  # Get pixel data and rescale it with slope
  RawDicom[:,:,currZ, currT] = currRef.pixel_array
  RSDicom[:, :, currZ, currT] = currRef.pixel_array / float(currRef.RescaleSlope)

# This is where I tried to 
rsArrayDicom = np.transpose(RSDicom, (1,0,2,3))
ArrayDicom = np.transpose(RawDicom, (1,0,2,3))

In [ ]:
# test output

print('rsArrayDicom:', rsArrayDicom.shape)
print('RescaleSlope:', currRef.RescaleSlope)
print('ArrayDicom:', ArrayDicom.shape)

In [ ]:
# # use only if RescaleSlope is NOT 1

# f,axarr = plt.subplots(1,2)
# a = axarr[0].imshow(rsArrayDicom[:,:,60,20], vmin=0,vmax=500)
# b = axarr[1].imshow(og_arr[:,:,60,20],vmin=0,vmax=500)
# plt.colorbar(a,cax=f.add_axes([0.92,0.2,0.02,0.6]))
# #plt.subplots_adjust(left=0,right=1.3)

In [ ]:
#Receive input for frames
print('Enter start frame (0 to',len(tVals)-1 , 'available): ')
frame1 = int(input())

print('Enter end frame (0 to',len(tVals)-1 , 'available): ')
frame2 = int(input())
print('Calculating frames',frame1,'to',frame2,'(inclusive)')

In [ ]:
# all time is in s
summed_arr = np.zeros(og_arr.shape[0:3])
sum_dur = 0
sum_dur_list = []

for i in np.arange(frame1, frame2+1):
  frameDuration = int(time2dur[tVals[i]])/1000
  summed_arr += (og_arr[:,:,:,i] * frameDuration)
  sum_dur += frameDuration
  sum_dur_list.append(sum_dur)

avg_arr = summed_arr / sum_dur

print('sum_dur_list:', sum_dur_list)
print('avg_arr:', avg_arr.shape)

In [ ]:
f,axarr = plt.subplots(1,2)
a = axarr[0].imshow(avg_arr[:,:,60], vmin=0,vmax=550)
b = axarr[1].imshow(ans_avg_arr[:,:,60,0],vmin=0,vmax=550)
plt.colorbar(a,cax=f.add_axes([0.92,0.2,0.02,0.6]))
plt.title('Calculated Average Slice vs. Original Average Slice', horizontalalignment='right')
#plt.subplots_adjust(left=0,right=1.3)

In [ ]:
# convert Numpy array --> Nifti
avg_nii = nb.Nifti1Image(avg_arr, og_image.affine)

In [ ]:
# Saver

print('Save? (Y/N): ')
inp = input().capitalize()
if inp =='Y':
  avg_name = sample_og_name.replace('PET', 'PET_avg40-90min')
  nb.save(avg_nii, rslt_pth + avg_name)
  print(avg_name + 'saved in ' + rslt_pth)